# Como usar

1. **Selecione as soluções a serem plotadas**: além da solução analítica, existem 3 outras soluções possíveis por 3 métodos numéricos diferentes
|Método|Nomes Alternativos|Erro proporcional a|Aproximação para este caso específico|
|:-:|:-:|:-:|:-|
|Método de Euler|Runge-Kutta de ordem 1 (RK1)|$$ \Delta t $$|$$u(t+\Delta t)\approx u(t) + \color{red}{u(t) \Delta t}$$|
|Runge-Kutta de ordem 2 (RK2)|Método de Euler Melhorado, Método de Heun|$$ \Delta t^2$$|$$u(t+\Delta t)\approx u(t) +\color{red}{ u(t)\Delta t}+\color{blue}{u(t)\frac{\Delta t^2}{2}}$$|
|Runge-Kutta de ordem 4 (RK4)| - |$$ \Delta t^4$$| $$u(t+\Delta t)\approx u(t) +\color{red}{u(t)\Delta t}+\color{blue}{u(t)\frac{\Delta t^2}{2}}+\color{orange}{u(t)\frac{\Delta t^3}{6}}+\color{magenta}{u(t)\frac{\Delta t^4}{24}}$$|

O fato de que os termos de ordem maior parecem ter saído de uma série de potências não é coincidência. Para mais detalhes sobre o método de Runge-Kutta, veja esta referência: [(Marcos Eduardo Valle, UNICAMP)](https://www.ime.unicamp.br/~valle/Teaching/MS211/Aula21.pdf).


2. **Selecione $N$**: o número de pontos $N$ utilizado para discretizar o eixo $x$ (que aqui representa o tempo) está intimamente relacionado com a precisão e o espaçamento $\Delta t$ entre dois pontos $t_i$ e $t_{i+1}$. Quanto maior $N$, mais as soluções numéricas se aproximarão da solução analitica e mais custo computacional será necessário para obte-lás.
***

In [36]:
%matplotlib widget
from ipywidgets import interact
import ipywidgets as wid
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

lay=wid.Layout(width="repeat(auto-fill, 180px)")
laybox=wid.Layout(width='95%')
layauto=wid.Layout(width="25%")
lay45=wid.Layout(width="30%")
laybmargin=wid.Layout(border='solid 1px black',
        margin='10px 0px 10px 0px',
        padding='5px 5px 5px 5px')

gridshow=wid.Checkbox(value=True, description='Mostrar Grid')
t12show=wid.Checkbox(value=True, description='Destacar meia vida')
OUTHBOX=wid.HBox([gridshow,t12show],layout=laybox)
outt=wid.Output(layout=laybmargin)

ELEMENTOS=['5730','163','12.35','15700000','183','24065','1600',
      '703800000','4468000000']
elementos=[('Carbono-14 (5 730 anos)',0),
            ('Cálcio-45 (163 dias)',1),
            ('Hidrogênio-3 (12.35 anos)',2),
            ('Iodo-129 (15 700 000 anos)',3),
            ('Ouro-195 (183 dias)',4),
            ('Plutônio-239 (24 065 anos)',5),
            ('Rádio-226 (1600 anos)',6),
            ('Urânio-235 (703 800 000 anos)',7),
            ('Urânio-238 (4 468 000 000 anos)',8)]

EL=wid.Dropdown(
    options=elementos,
    value=0,
    description='Material:',
    disabled=False,layout=lay45)
t12=wid.Text(
    value='5730',
    description='$t_{1/2}$:',layout=lay45)
u0=wid.FloatText(
    value=6,
    description='$u_0$:',layout=lay45)
N=wid.IntSlider(value=100,min=5,
    max=1000,step=5,
    description='$N$:',
    continuous_update=False,layout=laybox)
an=wid.ToggleButton(
    value=True,
    description='Analítico',
    tooltip='Ativa ou desativa a solução analítica',
    icon='check',layout=layauto)
e=wid.ToggleButton(
    value=False,
    description='RK1(Euler)',
    tooltip='Ativa ou desativa a solução RK1(Euler)',
    icon='check',layout=layauto)
r2=wid.ToggleButton(
    value=False,
    description='RK2',
    tooltip='Ativa ou desativa a solução RK2',
    icon='check',layout=layauto)
r4=wid.ToggleButton(
    value=True,
    description='RK4',
    tooltip='Ativa ou desativa a solução RK4',
    icon='check',layout=layauto)
with outt:
    fig, ax=plt.subplots(constrained_layout=True)

fig.canvas.toolbar_position='bottom'
fig.canvas.header_visible = False
fig.canvas.footer_visible = False

def main(u0,t12,N,an,e,r2,r4,gridshow,t12show):
    butchange()
    t12=eval(t12)
    t, dt=np.linspace(0,t12*4,N,retstep=True)
    ta=np.linspace(0,t12*4,1000)
    u02=u0/2
    a=np.log(2)/t12
    nt=t.size
    ue=np.zeros((nt))
    ur2=np.zeros((nt))
    ur4=np.zeros((nt))
    ue[0], ur2[0], ur4[0] = (u0,u0,u0)
    
    @njit
    def Fl(fv):
        return -a*fv
    
    @njit
    def euler(f,dt):
        for i in range(0,nt-1):
            f[i+1]=f[i]+Fl(f[i])*dt
        return f
    
    @njit
    def rk2(f,dt):
        for i in range(0,nt-1):
            k1=Fl(f[i])
            k2=Fl(f[i]+dt*k1)
            f[i+1]=f[i]+dt*(k1+k2)/2
        return f
    
    @njit
    def rk4(f,dt):
        for i in range(0,nt-1):
            k1=Fl(f[i])
            k2=Fl(f[i]+k1*dt/2)
            k3=Fl(f[i]+k2*dt/2)
            k4=Fl(f[i]+k3*dt)
            f[i+1]=f[i]+dt*(k1+2*k2+2*k3+k4)/6
        return f
    
    if e==True: ue=euler(ue,dt)
    if r2==True: ur2=rk2(ur2,dt)
    if r4==True: ur4=rk4(ur4,dt)
    with outt:
        fig=plt.clf()
        fig=plt.xlabel('Tempo (mesmas unidades de $t_{1/2}$)')
        fig=plt.ylabel('Quantidade (mesmas unidades de $u_0$)')
        if (e,an,r2,r4)!=(False,False,False,False):
            if an==True: fig=plt.plot(ta,u0*np.exp(-a*ta),
                            'k-',linewidth=6,label='Analítico')
            if r4==True: fig=plt.plot(t,ur4,
                            'o--',color='skyblue',label=f'RK4')
            if r2==True: fig=plt.plot(t,ur2,
                            'o--',color='fuchsia',label=f'RK2')
            if e==True: fig=plt.plot(t,ue,
                            'o--',color='coral',label=f'Euler') 
            tmax=t.max()
            fig=plt.ylim(0,u0+u0*0.1)
            fig=plt.xlim(-t12*0.1,tmax+t12*0.1)
            xtiks=np.arange(0,tmax+t12,t12)
            ytiks=np.arange(0,u0+u02,u02/2)
            #fig=plt.xticks([0,t12,tmax], ['0','$t_{1/2}$',f'{tmax:.2e}'])
            #fig=plt.yticks([0,u02,u0])
            fig=plt.yticks(ytiks,ytiks)
            fig=plt.xticks(xtiks,xtiks)
            if t12show==True:
                fig=plt.gca().get_xticklabels()[1].set_color('red')
                fig=plt.gca().get_yticklabels()[2].set_color('red')
                fig=plt.plot((-t12*0.1,t12),(u02,u02),'r--')#linha horizontal
                fig=plt.plot((t12,t12),(0,u02),'r--') #linha vertical
            fig=plt.title(f'$\Delta t =${dt:.2f} | $N=${N}')
            if gridshow==True:
                fig=plt.grid()
            leg=plt.legend()
        else:
            fig=plt.title('Nada para plotar')

def dropchange(s):
    t12.value=ELEMENTOS[s.new]
def butchange():
    if r4.value==True:
        r4.button_style='success'
    else:
        r4.button_style=''
    if r2.value==True:
        r2.button_style='success'
    else:
        r2.button_style=''
    if e.value==True:
        e.button_style='success'
    else:
        e.button_style=''
    if an.value==True:
        an.button_style='success'
    else:
        an.button_style=''
Hini=wid.HBox([u0,t12,EL])         
Hsol=wid.HBox([an,e,r2,r4],layout=laybox)
OPT=wid.VBox([Hini,N,Hsol])

titlegraph=wid.Accordion(children=[OPT,OUTHBOX])
titlegraph.set_title(0,'Opções do Sistema:')
titlegraph.set_title(1,'Opções do Gráfico:')
ALL=wid.VBox([titlegraph,outt])
out=wid.interactive_output(main, {'t12show':t12show,'gridshow':gridshow,'u0':u0,'t12':t12,'N':N,'an':an,'e':e,'r2':r2,'r4':r4})
display(ALL,out)

EL.observe(dropchange, names='value')

Output()

# Precisão (interativo)

Soluções numéricas e analíticas da equação para o decaimento radioativo de um material

$$\frac{du}{dt}=-au$$

As soluções plotadas são para os casos
$$u(t_0=0)=$$

$$t=[0, \, 2 \, t_{1/2}]$$